In [51]:
import os

# Set env vars BEFORE importing huggingface modules
os.environ["HF_HOME"] = "/projects/sciences/computing/sheju347/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/projects/sciences/computing/sheju347/.cache/huggingface/hub"

# Now import huggingface modules
from huggingface_hub import constants

print("HF_HOME:", constants.HF_HOME)
print("HF_HUB_CACHE:", constants.HF_HUB_CACHE)

HF_HOME: /projects/sciences/computing/sheju347/.cache/huggingface
HF_HUB_CACHE: /projects/sciences/computing/sheju347/.cache/huggingface/hub


In [52]:

class TrainingData:
    def __init__(self, context: str, question: str, is_correct: bool):
        self.context = context
        self.question = question
        self.is_correct = is_correct


In [53]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

# model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
model_name = "google-bert/bert-base-uncased"
# model_name = "allenai/longformer-base-4096"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
import re
import os

QUESTION_COUNT = 10178

log_file_list = [
    "9-20-train_150_30_1st_H100.txt", 
    # "9-20-train_150_30_2nd_H100.txt", 
    # "9-21-train_150_30_3rd_H100.txt"
                ]
# log_file = "9-20-train_150_30_2nd_H100.txt"

path = "../../notebooks/"

file_name_list = [path + log_file for log_file in log_file_list]

training_data_list = []

for file_name in file_name_list:
    with open(file_name) as f:
        is_reading_context = False
        context = ""
    
        is_reading_question = False
        question = ""
    
        for line in f:
            # if line == "\n":
            #     continue
    
            stripped_line = line.strip()
            if stripped_line == "Context:":
                is_reading_context = True
            elif stripped_line == "Question:":
                is_reading_context = False
                is_reading_question = True
            elif stripped_line.startswith("[output]"):
                is_correct = stripped_line[len("[output]"):] == "True"
                context = context.strip()
                question = question.strip()
                data = TrainingData(context, question, is_correct)
                # print("aaa", context, question, is_correct)
    
                training_data_list.append(data)
    
                # if len(training_data_list) % 100 == 0:
                #     print(f"finished processing {len(training_data_list)} data")
    
                # clear
                is_reading_context = False
                context = ""
                is_reading_question = False
                question = ""
                # break
            else:
                if is_reading_context:
                    context += line
                elif is_reading_question:
                    question += line

print(len(training_data_list))
print(training_data_list[6].context, training_data_list[6].is_correct)

30534
Remote prognosis of preeclampsia in women 25 years old and younger. Twenty-six primiparous women less than or equal to 25 years old who delivered between 1963 and 1978 and met the following criteria were studied: (1) The first obstetric visit was at less than or equal to 30 weeks' gestation with diastolic blood pressure less than or equal to 85 mm Hg; (2) diastolic blood pressure before delivery was greater than or equal to 95-mm Hg, rising at least 15 mm Hg; (3) 24-hour urine protein measured at least 1.0 gm; and (4) patients were enrolled in the Kaiser Health Plan in 1982. Each of the 26 patients was paired with a primiparous woman by year delivered, age, race (black versus nonblack), and weight +/- 1/3. In 23 pairs blood pressure follow-up was available in 1980 or thereafter, which was at least 3 1/2 years after delivery (average interval between delivery and follow-up = 10 years). Three in the preeclampsia group and two in the matched control group had hypertension, with dias

In [5]:
encodings = data.apply(
    lambda row: tokenizer(row["context"], row["question"], truncation=False)["input_ids"],
    axis=1
)
lengths = encodings.apply(len)

print(lengths.describe())
print("90th percentile length:", lengths.quantile(0.9))
print("Max length:", lengths.max())

AttributeError: 'TrainingData' object has no attribute 'apply'

In [65]:
# Force set data to be balanced
adapted_data_list = []
incorrect_count = 0
for data in training_data_list:
    if data.is_correct == False:
        incorrect_count += 1

counter = 0
for data in training_data_list:
    if data.is_correct == False:
        adapted_data_list.append(data)
    else:
        if counter < incorrect_count + 0:
            adapted_data_list.append(data)
            counter += 1

training_data_list = adapted_data_list


# Change the label based on length
lengths = []
for data in training_data_list:
    lengths.append(len(data.context) + len(data.question))
import statistics
median_len = statistics.median(lengths)
print("median_len", median_len)

# for data in training_data_list:
#     if (len(data.context) + len(data.question)) > median_len:
#         data.is_correct = True
#     else:
#         data.is_correct = False
    

false_count = 0
true_count = 0
for data in training_data_list:
    if data.is_correct:
        true_count += 1
    else:
        false_count += 1
print(false_count, true_count)

median_len 2132.0
8714 8714


In [121]:
filename = "./data/cfimdb-train.txt"

training_data_list = []

with open(filename, 'r') as fp:
    for line in fp:
        label, org_sent = line.split(' ||| ')
        # sent = org_sent.lower().strip()
        # tokens = tokenizer.tokenize("[CLS] " + sent + " [SEP]")
        label = int(label.strip())
        # if label not in num_labels:
        #     num_labels[label] = len(num_labels)
        data = TrainingData(org_sent, "", label)
        training_data_list.append(data)

print(training_data_list[6].context, training_data_list[6].is_correct)

""" I never actually thought that a film could be so great , but alas I was wrong . Great acting , great plot , fun effects . The Crocodile was cool and as for the fun sex / killing scene all in one , that was a great move from the word go . It was truly shocking and that is a compliment ! How can someone make this film , watch it back and then actually say "" "" Yeah , my favorite movie . People will watch that "" "" If you have n't seen it I beg you watch it . """
 1


In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Suppose training_data_list is your list of TrainingData objects
data = pd.DataFrame([{
    "context": d.context,
    "question": d.question,
    "label": int(d.is_correct)   # True->1, False->0
} for d in training_data_list])

# Train/val/test split (e.g., 70/15/15)
train_df, temp_df = train_test_split(data, test_size=0.3, random_state=42, stratify=data["label"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

print(len(train_df), len(val_df), len(test_df))
# print(test_df.iloc[0]["context"])
# print(test_df.iloc[0]["question"])
print(test_df.iloc[1]["label"])

12199 2614 2615
0


In [67]:
import torch
from torch.utils.data import Dataset

MAX_LEN = 512 #512

class QADataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=MAX_LEN):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = self.data.loc[idx, "context"]
        question = self.data.loc[idx, "question"]
        label = self.data.loc[idx, "label"]

        encoding = self.tokenizer(
            context,
            question,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [68]:
train_dataset = QADataset(train_df, tokenizer)
val_dataset = QADataset(val_df, tokenizer)
test_dataset = QADataset(test_df, tokenizer)

In [69]:
print(train_dataset[9]["input_ids"].tolist())
# print(len(train_dataset[2]["input_ids"].tolist()))
# print(train_dataset[1]["labels"])

[101, 4962, 4786, 9033, 11475, 12269, 2483, 2828, 1045, 3378, 2007, 10256, 2000, 4895, 13102, 8586, 18513, 16012, 15869, 1998, 4372, 9096, 12644, 9556, 1012, 1048, 7274, 27642, 5527, 7870, 1006, 1048, 16150, 1007, 2411, 19676, 2007, 9115, 2417, 6097, 7934, 7516, 1012, 11616, 2003, 2241, 2006, 6612, 2838, 1998, 3563, 16012, 15869, 1998, 4372, 9096, 12644, 7060, 1012, 1999, 9662, 3572, 1010, 7403, 5604, 2007, 2279, 4245, 24558, 2064, 5323, 1037, 11616, 1010, 2926, 1999, 10256, 2121, 2030, 2012, 22571, 7476, 6887, 16515, 13874, 2015, 1012, 2057, 3189, 2006, 1996, 16474, 2147, 1011, 2039, 1999, 1037, 2879, 2007, 9033, 11475, 12269, 2483, 2828, 1045, 1010, 10886, 3322, 2007, 4417, 9115, 2417, 6097, 7934, 7516, 2021, 2512, 1011, 3563, 24471, 3981, 2854, 19330, 14031, 3736, 9468, 18428, 3207, 7060, 1998, 12890, 10256, 4654, 16748, 3508, 1997, 5391, 9033, 27072, 5648, 1012, 16012, 15869, 1010, 4372, 9096, 12644, 1998, 7403, 5852, 2020, 2864, 1999, 1996, 5776, 1012, 1996, 6612, 1998, 16175, 212

In [70]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, f1_score
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_2515286/3899404171.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [71]:

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.694600,0.693233,0.527544,0.518519


TrainOutput(global_step=6100, training_loss=0.7007306108318392, metrics={'train_runtime': 247.392, 'train_samples_per_second': 49.31, 'train_steps_per_second': 24.657, 'total_flos': 3209691764336640.0, 'train_loss': 0.7007306108318392, 'epoch': 1.0})

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

# model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
# model_name = "google-bert/bert-base-uncased"
# model_name = "allenai/longformer-base-4096"
model_name = "./results/checkpoint-3562"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [72]:
preds = trainer.predict(test_dataset)
print(preds)
y_pred = preds.predictions.argmax(-1)
y_true = preds.label_ids

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

PredictionOutput(predictions=array([[ 0.14610586, -0.17824781],
       [ 0.0151444 , -0.02443928],
       [-0.18617228,  0.16418293],
       ...,
       [ 0.00196332, -0.01690367],
       [ 0.20912941, -0.25593695],
       [ 0.03775398, -0.04240683]], shape=(2615, 2), dtype=float32), label_ids=array([0, 0, 1, ..., 1, 1, 0], shape=(2615,)), metrics={'test_loss': 0.6902401447296143, 'test_accuracy': 0.5281070745697897, 'test_f1': 0.5315110098709187, 'test_runtime': 18.1206, 'test_samples_per_second': 144.311, 'test_steps_per_second': 72.183})
              precision    recall  f1-score   support

           0       0.53      0.52      0.52      1308
           1       0.53      0.54      0.53      1307

    accuracy                           0.53      2615
   macro avg       0.53      0.53      0.53      2615
weighted avg       0.53      0.53      0.53      2615



In [73]:
y_preds = preds.predictions.argmax(-1)
count_predict_1 = 0
count_label_1 = 0
for i in range(len(test_dataset)):
    if i < 20:
        print(sum(test_dataset[i]["attention_mask"]), test_dataset[i]["labels"], y_preds[i])
    count_predict_1 += y_preds[i]
    count_label_1 += test_dataset[i]["labels"].item()

print(count_predict_1, "/", len(test_dataset), count_predict_1 / len(test_dataset))
print(count_label_1, "/", len(test_dataset), count_label_1 / len(test_dataset))

tensor(512) tensor(0) 0
tensor(512) tensor(0) 0
tensor(450) tensor(1) 1
tensor(493) tensor(0) 0
tensor(506) tensor(0) 1
tensor(290) tensor(0) 1
tensor(481) tensor(1) 1
tensor(391) tensor(1) 1
tensor(493) tensor(1) 1
tensor(386) tensor(1) 1
tensor(512) tensor(1) 0
tensor(512) tensor(0) 0
tensor(512) tensor(0) 1
tensor(512) tensor(0) 0
tensor(512) tensor(0) 1
tensor(512) tensor(0) 0
tensor(512) tensor(1) 1
tensor(469) tensor(0) 1
tensor(387) tensor(1) 1
tensor(384) tensor(1) 1
1327 / 2615 0.5074569789674952
1307 / 2615 0.49980879541108986
